<a href="https://colab.research.google.com/github/sholaremu/sholaremu/blob/main/Feature_engineering_on_categorical_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np 

In [ ]:
vg_df = pd.read_csv('vgsales.csv', encoding='utf-8')
vg_df[['Name', 'Platform', 'Year', 'Genre', 'Publisher']].iloc[1:7]

In [ ]:
genres = np.unique(vg_df['Genre'])
genres

In [ ]:
from sklearn.preprocessing import LabelEncoder

gle = LabelEncoder()
genre_labels = gle.fit_transform(vg_df['Genre'])
genre_mappings = {index: label for index, label in enumerate(gle.classes_)}
genre_mappings

In [ ]:
vg_df['GenreLabel'] = genre_labels
vg_df[['Name', 'Platform', 'Year', 'Genre', 'GenreLabel']].iloc[1:7]

In [ ]:
poke_df = pd.read_csv('pokemon.csv', encoding='utf-8')
poke_df = poke_df.sample(random_state=1, frac=1).reset_index(drop=True)

np.unique(poke_df['generation'])

In [ ]:
gen_ord_map = {'gen 1': 1, 'gen 2': 2, 'gen 3': 3, 'gen 4': 4, 'gen 5': 5, 'gen 6': 6, 'gen 7': 7}

poke_df['generationlabel'] = poke_df['generation'].map(gen_ord_map)
poke_df[['name', 'generation', 'generationlabel']].iloc[4:10]

In [ ]:
poke_df[['name', 'generation', 'is_legendary']].iloc[4:10]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# transform and map pokemon generations
gen_le = LabelEncoder()
gen_labels = gen_le.fit_transform(poke_df['generation'])
poke_df['gen_label'] = gen_labels

# transform and map pokemon legendary status
leg_le = LabelEncoder()
leg_labels = leg_le.fit_transform(poke_df['is_legendary'])
poke_df['Lgnd_Label'] = leg_labels

poke_df_sub = poke_df[['name', 'generation', 'gen_label', 'is_legendary', 'Lgnd_Label']]
poke_df_sub.iloc[4:10]

In [ ]:
# encode generation labels ussing one-hot encoding scheme
gen_ohe = OneHotEncoder()
gen_feature_arr = gen_ohe.fit_transform(poke_df[['gen_label']]).toarray()
gen_feature_labels = list(gen_le.classes_)
gen_features = pd.DataFrame(gen_feature_arr, columns=gen_feature_labels)

# encode legendary status labels using one-hot encoding scheme
leg_ohe = OneHotEncoder()
leg_feature_arr = leg_ohe.fit_transform(poke_df[['Lgnd_Label']]).toarray()
leg_feature_labels = ['Legendary_'+str(cls_label) for cls_label in leg_le.classes_]
leg_features = pd.DataFrame(leg_feature_arr, columns=leg_feature_labels)

In [ ]:
poke_df_ohe =pd.concat([poke_df_sub, gen_features, leg_features], axis=1)
columns = sum([['name', 'generation', 'gen_label'], gen_feature_labels,['is_legendary', 'Lgnd_Label'],
              leg_feature_labels], [])
poke_df_ohe[columns].iloc[4:10]

In [ ]:
new_poke_df = pd.DataFrame([['PikaZoom', '3', True],
                            ['CharMyToast', '4', False]],
                           columns=['name', 'generation', 'is_legendary'])
new_poke_df

In [ ]:
new_gen_labels = gen_le.transform(new_poke_df['generation'])
new_poke_df['Gen_Label'] = new_gen_labels

new_leg_labels = leg_le.transform(new_poke_df['is_legendary'])
new_poke_df['Lgnd_Label'] = new_leg_labels

new_poke_df[['name', 'generation', 'Gen_Label', 'is_legendary', 'Lgnd_Label']]

In [ ]:
new_gen_feature_arr = gen_ohe.transform(new_poke_df[['Gen_Label']]).toarray()
new_gen_features = pd.DataFrame(new_gen_feature_arr, columns=gen_feature_labels)

new_leg_feature_arr = leg_ohe.transform(new_poke_df[['Lgnd_Label']]).toarray()
new_leg_features = pd.DataFrame(new_leg_feature_arr, columns=leg_feature_labels)

new_poke_ohe = pd.concat([new_poke_df, new_gen_features, new_leg_features], axis=1)
columns = sum([['name', 'generation', 'Gen_Label'], gen_feature_labels,
               ['is_legendary', 'Lgnd_Label'], leg_feature_labels], [])
new_poke_ohe[columns]

In [ ]:
gen_onehot_features =pd.get_dummies(poke_df['generation'])
pd.concat([poke_df[['name', 'generation']], gen_onehot_features], axis=1)


In [ ]:
gen_dummy_features = pd.get_dummies(poke_df['generation'], drop_first=True)
pd.concat([poke_df[['name', 'generation']], gen_dummy_features], axis=1).iloc[4:10]

In [ ]:
gen_onehot_features = pd.get_dummies(poke_df['generation'])
gen_dummy_features = gen_onehot_features.iloc[:, :-1]
pd.concat([poke_df[['name', 'generation']], gen_dummy_features], axis=1).iloc[4:10]

In [ ]:
gen_onehot_features = pd.get_dummies(poke_df['generation'])
gen_effect_features = gen_onehot_features.iloc[:,:-1]
gen_effect_features.loc[np.all(gen_effect_features == 0, axis=1)] = -1.
pd.concat([poke_df[['name', 'generation']], gen_effect_features], axis=1).iloc[4:10]

In [ ]:
unique_genres = np.unique(vg_df[['Genre']])
print("Total game genres:", len(unique_genres))
print(unique_genres)

In [ ]:
from sklearn.feature_extraction import FeatureHasher

fh = FeatureHasher(n_features=6, input_type='string')
hashed_features = fh.fit_transform(vg_df['Genre'])
hashed_features = hashed_features.toarray()
pd.concat([vg_df[['Name', 'Genre']], pd.DataFrame(hashed_features)], axis=1).iloc[1:7]